In [5]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import re
import openpyxl
import datetime

### SUBIR PADRON SSS

In [6]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo =  archivo.name # es para obtner la direccion correcta del archivo con extencion.
        columnas = [    "Código de Obra Social",
                        "CUIT del empleador",
                        "CUIL del Titular",
                        "Código de Parentesco",
                        "Cuil",
                        "Tipo Documento",
                        "Número Documento",
                        "Apellido y Nombre",
                        "Sexo",
                        "Estado Civil",
                        "Fecha de Nacimiento",
                        "Nacionalidad",
                        "Calle",
                        "Número de puerta",
                        "Piso",
                        "Departamento",
                        "Localidad",
                        "Código Postal",
                        "Provincia",
                        "Tipo de Domicilio",
                        "Teléfono",
                        "Situación revista",
                        "Incapacidad",
                        "Tipo de beneficiario titular",
                        "Fecha de alta en la Obra Social",
                        "Fecha de cierre de la presentación",
                        "Valida CUIL",
                        "CUIL informado por la OS",
                        "Tipo beneficiario informado en la DDJJ (Declaración Jurada del empleador)",
                        "CUIT empleador informante en la DDJJ",
                        "Obra Social declarada en la DDJJ",
                        "Ultimo período declarado por el empleador en la DDJJ",
                        "Obra Social elegida (en caso de opción)",
                        "Período desde de vigencia de la opción"
                        ]
        try:
            padron_sss = pd.read_csv(ruta_archivo, sep='|', names=columnas)

            print (padron_sss)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/comparacion_padrones/127406-202402-PC.txt' mode='r' encoding='cp1252'>
       Código de Obra Social  CUIT del empleador  CUIL del Titular  \
0                     127406         20120810805       27264313061   
1                     127406         30714444162       20214552478   
2                     127406         30570955388       20246881422   
3                     127406         30714444162       20214552478   
4                     127406         30676122601       20134647494   
...                      ...                 ...               ...   
13998                 127406         27344614038       27344614038   
13999                 127406         27949506652       27949506652   
14000                 127406         27949509902       27949509902   
14001                 127406         27949519886       27949519886   
14002                 127406         27949519886       27949519886   

       Código de Parentesco         Cu

## cargar padron de sidefi

In [7]:
archivo = None
while True:
    archivo = funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            #padron = pd.read_excel(ruta_archivo)
            padron = pd.read_csv(archivo, sep=";", encoding="ISO-8859-1")
            print(padron)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

<_io.TextIOWrapper name='C:/Users/zickd/Downloads/comparacion_padrones/8089 - copia (2).csv' mode='r' encoding='cp1252'>
       Nro. OS  Plan del Afiliado Opcion  Nro. Grupo Familiar  \
0            7                 10     SI             26407599   
1            7                 10     SI             26407599   
2            7                 10     SI             26407599   
3           10                 10     SI             31403090   
4           15                100     NO             35995281   
...        ...                ...    ...                  ...   
12629        4                 10     SI             43194699   
12630        4                 10     SI             43194699   
12631       14                100     NO             31894447   
12632        1                100     SI             36602782   
12633        7                 10     SI             32749818   

         Nro. Afiliado Parentesco con el TITULAR Apellido Afiliado  \
0      020-26407599-00      

In [8]:
padron.columns


Index(['Nro. OS', 'Plan del Afiliado', 'Opcion', 'Nro. Grupo Familiar',
       'Nro. Afiliado', 'Parentesco con el TITULAR', 'Apellido Afiliado',
       'Nombre Afiliado', 'Tipo Documento', 'Nro. Documento', 'Nro. CUIL',
       'Fecha Nacimiento', 'Sexo', 'Nacionalidad', 'Estado Civil',
       'Legajo Interno', 'Nro. Cuenta', 'Fecha Alta', 'Fecha Baja',
       'Observacion Baja', 'Tipo Afiliado', 'Dependencia', 'Observacion',
       'Calle', 'Nro. Puerta', 'Piso', 'Departamento', 'Barrio', 'Localidad',
       'Provincia', 'Codigo Postal', 'Telefono', 'Celular', 'Email'],
      dtype='object')

###  transformar dependencia y Nr OS en sidefi 

In [9]:
padron['Dependencia']=padron['Dependencia'].astype(str)

dependencias = {
"10" : "CABA",
"20" : "ZONA NORTE Y NOROESTE",
"30" : "ZONA OESTE",
"40" : "ZONA SUR",
"50" : "LA PLATA",
"60" : "MAR DEL PLATA",
"70" : "PERGAMINO",
"80" : "ROSARIO",
"90" : "CORDOBA",
"100" : "MENDOZA",
"110" : "TUCUMAN"
}

convenios ={
    1 : "osoetsylra",
    4 : "SD",
    5 : "TIS",
    7 : "EVP",
    10 : "MEPLIFE",
    14: "osoetsylra",
    15: "osoetsylra",
}

padron['Dependencia']=padron['Dependencia'].replace(dependencias)
padron['Dependencia'].head(5)
padron['Nro. OS']=padron['Nro. OS'].replace(convenios)



## Cruzar los datos 

In [10]:
cruce_sidefi = pd.merge(padron, padron_sss, left_on='Nro. Documento', right_on='Número Documento', how='outer', indicator=True)
cruce_sidefi
save_path = filedialog.askdirectory()
cruce_sidefi.to_excel(save_path + '\cruce_sss.xlsx', engine='openpyxl' ,index = False )
cruce_sidefi.columns

Index(['Nro. OS', 'Plan del Afiliado', 'Opcion', 'Nro. Grupo Familiar',
       'Nro. Afiliado', 'Parentesco con el TITULAR', 'Apellido Afiliado',
       'Nombre Afiliado', 'Tipo Documento_x', 'Nro. Documento', 'Nro. CUIL',
       'Fecha Nacimiento', 'Sexo_x', 'Nacionalidad_x', 'Estado Civil_x',
       'Legajo Interno', 'Nro. Cuenta', 'Fecha Alta', 'Fecha Baja',
       'Observacion Baja', 'Tipo Afiliado', 'Dependencia', 'Observacion',
       'Calle_x', 'Nro. Puerta', 'Piso_x', 'Departamento_x', 'Barrio',
       'Localidad_x', 'Provincia_x', 'Codigo Postal', 'Telefono', 'Celular',
       'Email', 'Código de Obra Social', 'CUIT del empleador',
       'CUIL del Titular', 'Código de Parentesco', 'Cuil', 'Tipo Documento_y',
       'Número Documento', 'Apellido y Nombre', 'Sexo_y', 'Estado Civil_y',
       'Fecha de Nacimiento', 'Nacionalidad_y', 'Calle_y', 'Número de puerta',
       'Piso_y', 'Departamento_y', 'Localidad_y', 'Código Postal',
       'Provincia_y', 'Tipo de Domicilio', 'Teléfo

In [11]:
cruce_sidefi['Tipo de beneficiario titular'].fillna(20, inplace=True)
cruce_sidefi['Tipo de beneficiario titular'].astype(int)


0         7
1         7
2         7
3        20
4         0
         ..
15469     7
15470     7
15471     5
15472     7
15473     7
Name: Tipo de beneficiario titular, Length: 15474, dtype: int32

###  modelado tabla final

In [12]:
# crear nuevos campos codem y monotributo

cruce_sidefi["CODEM"] = ""
cruce_sidefi["MONOTRIBUTO SSS"] = ""

cruce_sidefi['Dependencia'].head(5)
tipo_benef= {
0:"RELACION DE DEPENDENCIA",
1:"PASANTES",
2:"JUBILADOS DEL SISTEMA NACIONAL DEL SEGURO DE SALUD",
3:"ADHERENTES VOLUNTARIOS",
4:"PEQUEÑOS CONTRIBUYENTES – MONOTRIBUTISTAS (LEY 25.865)",
5:"SERVICIO DOMESTICO (LEY 26.068)",
6:"JUBILADOS DEL REGISTRO PARA LA ATENCION DE ESPAÑOLES",
7:"MONOTRIBUTISTAS EFECTORES SOCIALES",
8:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 24.013)",
9:"BENEFICIARIOS DE SEGURO DE DESEMPLEO (LEY 25.191) (RENATRE)",
10:"JUBILADOS Y PENSIONADOS DEL INSTITUTO NACIONAL DE SERVICIOS SOCIALES",
11:"JUBILADOS Y PENSIONADOS DE OTROS SISTEMAS",
12:"RELACION DE DEPENDENCIA (SIN APORTES Y CONTRIBUCION POR SJP)",
20:""
}
cruce_dic = {
    "left_only":"Sidefi",
    "both":"ambos",
    "right_only":"SSS"
    }
cruce_sidefi['_merge'].replace(cruce_dic, inplace=True)
cruce_sidefi.rename(columns={'_merge':'padron sidefi/sss'}, inplace= True)

cruce_sidefi['Tipo de beneficiario titular'].replace(tipo_benef, inplace=True)
cruce_sidefi['Tipo Afiliado'].replace(tipo_benef, inplace=True)

parentesco = {
0 : "titular",
1 : "conyuge",
2 : "concubino/a",
3 : "hijo soltero menor 21",
4 : "hijo soltero 21 a 25 cursando estudios",
5 : "hijo del conyuge soltero menor 21",
1 : "hijo del conyuge soltero 21 a 25 cursando estudios",
1 : "menor bajo guardia o tutela",
8 : "familiar a cargo",
9 : "mayor 25, discapacitado"
}

cruce_sidefi['Código de Parentesco'].replace(parentesco, inplace = True)

cruce_sidefi=cruce_sidefi[[
"Nro. OS",
"Plan del Afiliado",
"Parentesco con el TITULAR",
"Apellido Afiliado",
"Nombre Afiliado",
"Nro. Documento",
"Nro. CUIL",
"Fecha Nacimiento",
"Fecha Alta",
"Fecha Baja",
"Observacion Baja",
"Tipo Afiliado",
"Dependencia",
"Localidad_x",
"Provincia_x",
"CUIL del Titular",
"Código de Parentesco",
"Cuil",
"Número Documento",
"Apellido y Nombre",
"Sexo_y",
"Fecha de Nacimiento",
"Localidad_y",
"Código Postal",
"Provincia_y",
"Teléfono",
"Incapacidad",
"Tipo de beneficiario titular",
"Fecha de alta en la Obra Social",
"CUIL informado por la OS",
"Obra Social elegida (en caso de opción)",
"Período desde de vigencia de la opción",
"padron sidefi/sss",
"CODEM",
"MONOTRIBUTO SSS"
]]

### guardar el excel por  sede

In [13]:
ahora = datetime.datetime.now()    # Formatear la fecha y hora como una cadena con el formato "aaaa-mm-dd hh:mm:ss"
fecha = ahora.strftime("%Y-%m-%d %H-%M-%S")

groups = cruce_sidefi.groupby('Dependencia') # Agrupar el dataframe por la columna convenio
dfs = [] # Crear una lista vacía para almacenar los dataframes separados
for name, group in groups: # Iterar sobre los grupos y añadir cada dataframe a la lista
    dfs.append(group)
    print(dfs) # Imprimir la lista de dataframes

save_path = filedialog.askdirectory()
cruce_sidefi.to_excel(save_path + '\devoluciones2_septiembre2023.xlsx', engine='openpyxl' ,index = False )

### guardarlos con los nombres correspondientes
for df in dfs: # Iterar sobre la lista de dataframes
    sede = df['Dependencia'].iloc[0] # Obtener el nombre de la sede del primer elemento del grupo
    filename = save_path + '/' + sede + '.xlsx' # Crear el nombre del archivo con la ruta de la carpeta y el nombre del convenio
    df.to_excel(filename, index=False) # Guardar el dataframe en el archivo excel sin el índice
    print(f"Se ha guardado el archivo {filename}") # Imprimir un mensaje de confirmación

[          Nro. OS  Plan del Afiliado Parentesco con el TITULAR  \
4              15              100.0                   TITULAR   
5              15              100.0                    Hijo/a   
20     osoetsylra              100.0                   TITULAR   
37        MEPLIFE               10.0                   TITULAR   
54             SD               10.0                   TITULAR   
...           ...                ...                       ...   
12683  osoetsylra              100.0                   TITULAR   
12684          14              100.0                   TITULAR   
12686          SD               10.0                   TITULAR   
12693  osoetsylra              100.0                   TITULAR   
12694         EVP               10.0                   TITULAR   

      Apellido Afiliado       Nombre Afiliado  Nro. Documento     Nro. CUIL  \
4                 ABATE          GISELA NOELI      35995281.0  2.735995e+10   
5              FRITAYON        TOBIAS TIZIANO   

### enviar cada excel por mail a las sedes